In [6]:
import socket
from pyspark.sql import SparkSession

driver_ip=socket.gethostbyname(socket.gethostname())

minio_endpoint=socket.gethostbyname('minio.minio.svc.cluster.local')

print(driver_ip)

print(minio_endpoint)

spark = SparkSession.builder \
 .master("k8s://https://192.168.10.101:6443") \
 .config("spark.driver.host", driver_ip) \
 .config("spark.kubernetes.container.image", "nexus.davidgreco.it:5000/spark-py:2.4.5") \
 .config("spark.kubernetes.namespace", "spark") \
 .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark") \
 .config("spark.kubernetes.container.image.pullSecrets", "regcred") \
 .config("spark.kubernetes.executor.request.cores", "0.5") \
 .config("spark.kubernetes.executor.label.role", "spark-node-group") \
 .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.7,io.delta:delta-core_2.11:0.6.1,org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4,org.apache.spark:spark-avro_2.11:2.4.4") \
 .config("spark.hadoop.fs.s3a.endpoint", "http://" + minio_endpoint + ":9000") \
 .config("spark.hadoop.fs.s3a.access.key", "AKaHEgQ4II0S7BjT6DjAUDA4BX") \
 .config("spark.hadoop.fs.s3a.secret.key", "SKFzHq5iDoQgF7gyPYRFhzNMYSvY6ZFMpH") \
 .config("spark.hadoop.fs.s3a.path.style.access", "true") \
 .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
 .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
 .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
 .config("spark.executor.instances", "4") \
 .config("spark.executor.memory", "4G") \
 .config("spark.executor.cores", "4") \
 .getOrCreate()

10.36.0.7
10.103.204.82


In [7]:
taxidata = spark.read.parquet("s3a://home/data-sets/nyc-taxi")

In [8]:
taxidata.createTempView("taxidata")

In [9]:
#spark.sql("""SELECT passenger_count, substring(pickup_datetime,1,4) trip_year, round(trip_distance), count(*) trips FROM taxidata GROUP BY passenger_count, substring(pickup_datetime,0,4), round(trip_distance) ORDER BY trip_year, trips desc""").show()
spark.sql("SELECT substring(pickup_datetime,1,4) as trip_year, * FROM taxidata order by trip_year desc").show()

KeyboardInterrupt: 

In [ ]:
spark.stop()